# Importando Bilbliotecas
### Aqui Usaremos as libs spark necessárias para o desenvolviemnto. SparkSession, que permite a execução do código Spark ou no cluster ou standalone; Functions, dando acesso às várias funções e;  Types, pois trabalharemos com os tipos Spark

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

# Criação da sessão Spark
### Uso do sparkSession para criar o ponto de entrada do spark para se trabalhar com RDD´s, dataframes e Datasets

In [ ]:
spark = SparkSession \
    .builder \
    .appName("Exemplo de ETL CSV") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

# Pre-definindo o Schema 
### Uso do Schema pré-definido se torna Muito mais rápido do que deixar o spark fazer a inferência

In [ ]:
sch = StructType() \
        .add("data", StringType(),True) \
        .add("dia_semana", StringType(),True) \
        .add("operacao", StringType(),True) \
        .add("TMR", StringType(),True) \
        .add("solicitacao_site", StringType(),True) \
        .add("entregues_novos", StringType(),True) \
        .add("perc_entregues", StringType(),True) \
        .add("atendidos_novos", StringType(),True) \
        .add("entregues_reaprov", StringType(),True) \
        .add("atendidos_reaprov", StringType(),True) \
        .add("perc_atend_nr", StringType(),True) \
        .add("perc_atend_novos", StringType(),True) \
        .add("perc_ete_novos", StringType(),True) \
        .add("entregues_nr", StringType(),True) \
        .add("atendidos_nr", StringType(),True) \
        .add("perc_atend_total", StringType(),True) \
        .add("aband_novos", StringType(),True) \
        .add("aband_reaprov", StringType(),True) \
        .add("cham_curta_novos", StringType(),True) \
        .add("cham_curta_reaprov", StringType(),True)

# Leitura CSV
### Leitura do CSV especificando o Format, com cabeçalho, codificaçao, sshema e delimitadores.

In [ ]:
df = spark.read.format("csv") \
     .option("header", True) \
     .option("encoding","UTF-8") \
     .option("delimiter",";") \
     .schema(sch) \
     .load("/content/drive/MyDrive/ESTUDOS/CSV/relatorio15.csv")

# Data Cleaning
### Aqui faremos ajustes e limpezas no dataframe, como ajustes do campo data que vem em formato Brasileio TimeStamp, e retirada do caracter "%" de algumas colunas numéricas

### Criação de uma UDF (Função Definida por Usuário) para tratamento dos campos data, mudando a coluna de timestamp para o format yyyy-MM-dd

In [ ]:
@udf(returnType=StringType()) 
def udt_trat(str):
    return str[6:10]+'-'+str[3:5]+'-'+str[0:2]

df = df.withColumn('data', udt_trat(col('data')))

### Uso do Regexp_replace para retirar o caracter "%" de algumas colunas. Pode-se usar também  o Translat criando um novo dataframe (newDf = testDF.withColumn('d_id', translate('d_id', 'a', '0')))

In [ ]:
df = df.withColumn('perc_entregues', regexp_replace('perc_entregues','%',''))
df = df.withColumn('perc_atend_nr', regexp_replace('perc_atend_nr','%',''))
df = df.withColumn('perc_atend_novos', regexp_replace('perc_atend_novos','%',''))
df = df.withColumn('perc_ete_novos', regexp_replace('perc_ete_novos','%',''))
df = df.withColumn('perc_atend_total', regexp_replace('perc_atend_total','%',''))
